In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl

hl.init(
    tmp_dir='/net/ascratch/people/plggosborcz/gosborcz-hail',
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '30G'},
    default_reference='GRCh38') 

2023-01-09 16:06:45.244 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0075:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.105-acd89e80c345
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/analysis/burden-and-family/hail-20230109-1606-0.2.105-acd89e80c345.log


In [2]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook() 

Loading BokehJS ...

Loading BokehJS ...

## 1. Filtering for quality 

In [ ]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/resources/repeatmasker-extended-keyed.ht')

cov = hl.read_table('/net/archive/groups/plggneuromol/resources/gnomad/gnomad.genomes.r3.0.1.coverage.ht')
cov = cov.filter(cov.over_1 > 0.9)

mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-unfiltered.mt')
mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)

mt.checkpoint('/net/scratch/people/plggosborcz/temp-mts/rpmk.mt')

mt = hl.read_matrix_table('/net/scratch/people/plggosborcz/temp-mts/rpmk.mt')
mt = mt.naive_coalesce(10000)
mt.write('/net/scratch/people/plggosborcz/temp-mts/rpmk-repart-1.mt')

mt = hl.read_matrix_table('/net/scratch/people/plggosborcz/temp-mts/rpmk-repart-1.mt')
mt.count()

mt = mt.filter_rows(hl.is_defined(cov[mt.locus]), keep = True)
mt.checkpoint('/net/scratch/people/plggosborcz/temp-mts/gts-cov.mt')

mt = hl.read_matrix_table('/net/scratch/people/plggosborcz/temp-mts/gts-cov.mt')
mt = mt.naive_coalesce(1000)
mt.checkpoint('/net/scratch/people/plggosborcz/temp-mts/gts-cov-repart.mt')

mt = mt.annotate_rows(dp_qc = hl.agg.stats(mt.DP),
                     gq_qc = hl.agg.stats(mt.GQ),
                     hwe = hl.agg.hardy_weinberg_test(mt.GT))

mt = mt.annotate_rows(n_below_dp_3 = hl.agg.count_where(mt.DP < 3),
                      n_below_gq_30 = hl.agg.count_where(mt.GQ <30))

mt.checkpoint('/net/scratch/people/plggosborcz/temp-mts/qc-gts.mt')

mt = hl.read_matrix_table('/net/scratch/people/plggosborcz/temp-mts/qc-gts.mt')

mt = mt.filter_rows((mt.dp_qc.mean > 5) &
                    (mt.gq_qc.mean > 50) &
                    (mt.dp_qc.mean < 200) &
                    (mt.hwe.p_value > 0.05) &
                    (mt.n_below_dp_3 < 3) &
                    (mt.n_below_gq_30 < 10))

mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/gts-fams-filtered.mt')

## 2. Split multiallelic variants and annotate with gnomad and other databases

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/gts-fams-filtered.mt')
mt = mt.distinct_by_row()
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/to-delete.mt')

mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/to-delete.mt')

mt = mt.key_rows_by(mt.locus, mt.alleles)
mt = hl.split_multi_hts(mt)
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-split.mt')

gnomad = hl.read_table('/net/archive/groups/plggneuromol/resources/gnomad/gnomad.genomes.v3.1.1.sites.ht/')

mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-split.mt')
mt = mt.annotate_rows(gnomad_v_3_1 = gnomad[mt.row_key])

mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-gnomad.mt')

2022-05-15 20:53:54 Hail: INFO: wrote matrix table with 15394580 rows and 124 columns in 1000 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/to-delete.mt
2022-05-15 20:56:40 Hail: INFO: wrote matrix table with 15760027 rows and 124 columns in 1000 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-split.mt


In [4]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-gnomad.mt')

In [7]:
genes = hl.read_table('/net/archive/groups/plggneuromol/resources/genecode_v32.ht')
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))

hpo = hl.import_table('/net/archive/groups/plggneuromol/resources/hpo.tsv', impute = True, no_header=True)

start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)

genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])
genes = genes.key_by(genes.interval)

mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-gnomad.mt')

mt = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
mt = mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-anno.mt')

2022-05-16 09:39:03 Hail: INFO: Reading table to impute column types
2022-05-16 09:39:05 Hail: INFO: Finished type imputation
  Loading field 'f0' as type str (imputed)
  Loading field 'f1' as type str (imputed)
2022-05-16 09:39:06 Hail: INFO: Ordering unsorted dataset with network shuffle2]
2022-05-16 09:39:08 Hail: INFO: Ordering unsorted dataset with network shuffle1]
2022-05-16 09:39:09 Hail: INFO: Ordering unsorted dataset with network shuffle2]
2022-05-16 09:39:10 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-05-16 09:39:11 Hail: INFO: Ordering unsorted dataset with network shuffle1]
2022-05-16 09:39:11 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-05-16 09:46:34 Hail: INFO: wrote matrix table with 15760027 rows and 124 columns in 1000 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-anno.mt
    Total size: 41.48 GiB
    * Rows/entries: 41.48 GiB
    * Columns: 550.00 B
    * Globals: 11.00 B
    * Sm

## 3. Annotate with phenotypes + create a separate mt for each of the larger families

In [3]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-anno.mt')

In [4]:
pheno = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv', impute = True, key='ID', delimiter = ',', quote ="\"")

2022-05-17 13:25:18 Hail: INFO: Reading table to impute column types
2022-05-17 13:25:19 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading field 'kinship' as type str (imputed)
  Loading field 'disease' as type str (imputed)
  Loading field 'phenotype' as type str (imputed)
  Loading field 'add_pheno' as type str (imputed)
  Loading field 'heavy_tics' as type str (imputed)
  Loading field 'heavy_tics_familial' as type str (imputed)
  Loading field 'GTS_ASD_group' as type str (imputed)
  Loading field 'nonCTD' as type str (imputed)


In [5]:
mt = mt.annotate_cols(phenotypes = pheno[mt.s])

## 4. For each of the families get intragenic variants with cadd > 5 for further filtering

In [6]:
fams = list(set(mt.phenotypes.family.collect()))

In [7]:
fams.sort()

In [10]:
cadd = hl.read_table('/net/archive/groups/plggneuromol/resources/cadd.ht')

In [ ]:
for f in fams:
    fam = mt.filter_cols(mt.phenotypes.family == f)
    fam = fam.filter_rows(hl.agg.any(fam.GT.is_non_ref()))
    fam = fam.naive_coalesce(50)
    fam = fam.filter_rows(fam.within_gene == hl.empty_array(hl.tstr), keep = False)
    fam = fam.annotate_rows(cadd = cadd[fam.row_key])
    fam = fam.filter_rows(fam.cadd.score_phred > 5)
    fam.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-filtered'+f+'.mt')

2022-05-16 11:42:00 Hail: INFO: wrote matrix table with 343873 rows and 7 columns in 50 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-filteredA.mt
    Total size: 2.34 GiB
    * Rows/entries: 2.34 GiB
    * Columns: 166.00 B
    * Globals: 11.00 B
    * Smallest partition: 21 rows (125.68 KiB)
    * Largest partition:  9745 rows (68.39 MiB)
2022-05-16 11:56:59 Hail: INFO: wrote matrix table with 363663 rows and 8 columns in 50 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-filteredB.mt
    Total size: 2.47 GiB
    * Rows/entries: 2.47 GiB
    * Columns: 206.00 B
    * Globals: 11.00 B
    * Smallest partition: 31 rows (134.79 KiB)
    * Largest partition:  10002 rows (70.27 MiB)


### Additional annotation with vep:

In [8]:
vep = hl.read_table('/net/archive/groups/plggneuromol/resources/vep38/grch38_context_vep_annotated.ht')

In [ ]:
for f in fams:
    fam = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-filtered'+f+'.mt')
    fam = fam.annotate_rows(vep = vep[fam.row_key])
    fam.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-vep'+f+'.mt')

## 5. For each of the families find pathogenic variants according to mutation schema:

* first in all of the genes (maf < 0.001 or not in Gnomad, cadd > 10)
* then in selected genes (maf < 0.05 or not in Gnomad, cadd > 10) 
* ten in all of the genes (maf < 0.01 or not in Gnomad, cadd > 5)
* then in selected genes (maf < 0.1 or not in Gnomad, cadd > 5)

In [3]:
genes = pd.read_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/burden-and-family/all-genes-burde-and-family.csv', names=['gene'])

In [4]:
all_genes = set(genes.gene)
all_genes = list({x.replace('\xa0\xa0\xa0', '') for x in all_genes})

In [5]:
len(all_genes)

953

## 6. For each of the larger families find pathogenic variants according to the schema by KF

In [6]:
schema = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/burden-and-family/familiespattern.csv', impute = True, key='sample', delimiter = '\;', quote ="\"")

2022-06-22 21:14:07 Hail: INFO: Reading table to impute column types
2022-06-22 21:14:08 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']
2022-06-22 21:14:08 Hail: INFO: Finished type imputation
  Loading field 'sample' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'mutation_option_1' as type str (imputed)
  Loading field 'mutation_option_2' as type str (imputed)


## 6.1 Export a table with genes that are duplicated 
- this is first done withou the T and H families 
(these are double schema so they are dine in the end)

In [7]:
fams = ['A', 'B', 'C','D', 'E', 'F', 'G', 'I', 'J', 'R', 'S', 'U', 'W', 'X', 'Y'] 

fams_all_genes = ['A_all_genes', 'B_all_genes', 'C_all_genes','D_all_genes',
                  'E_all_genes', 'F_all_genes', 'G_all_genes', 
                  'I_all_genes', 'J_all_genes', 'R_all_genes', 'S_all_genes',
                  'U_all_genes', 'W_all_genes', 'X_all_genes', 'Y_all_genes']

fams_selected_genes = ['A_selected_genes', 'B_selected_genes', 'C_selected_genes','D_selected_genes',
                  'E_selected_genes', 'F_selected_genes', 'G_selected_genes',
                  'I_selected_genes', 'J_selected_genes', 'R_selected_genes', 'S_selected_genes',
                  'U_selected_genes', 'W_selected_genes', 'X_selected_genes', 'Y_selected_genes']

fams_all_genes_other_treshold = ['A_all_genes_lower_treshold', 'B_all_genes_lower_treshold',
                                 'C_all_genes_lower_treshold','D_all_genes_lower_treshold',
                  'E_all_genes_lower_treshold', 'F_all_genes_lower_treshold', 'G_all_genes_lower_treshold', 
                  'I_all_genes_lower_treshold', 'J_all_genes_lower_treshold', 'R_all_genes_lower_treshold',
                   'S_all_genes_lower_treshold','U_all_genes_lower_treshold', 'W_all_genes_lower_treshold',
                   'X_all_genes_lower_treshold', 'Y_all_genes_lower_treshold']

fams_selected_genes_other_treshold = ['A_selected_genes_lower_treshold', 'B_selected_genes_lower_treshold',
                                      'C_selected_genes_lower_treshold','D_selected_genes_lower_treshold',
                  'E_selected_genes_lower_treshold', 'F_selected_genes_lower_treshold', 'G_selected_genes_lower_treshold',
                  'I_selected_genes_lower_treshold', 'J_selected_genes_lower_treshold', 'R_selected_genes_lower_treshold',
                                      'S_selected_genes_lower_treshold',
                  'U_selected_genes_lower_treshold', 'W_selected_genes_lower_treshold', 'X_selected_genes_lower_treshold',
                                      'Y_selected_genes_lower_treshold']

fams_description = ['A_description', 'B_description', 'C_description','D_description',
                  'E_description', 'F_description', 'G_description',
                  'I_description', 'J_description', 'R_description', 'S_description',
                  'U_description', 'W_description', 'X_description', 'Y_description']

fams1 = []
fams2 = []
fams3 = []
fams4 = []
fams_to_export_all = []
fams_to_export_selected = []
fams_to_export_all_other_treshold = []
fams_to_export_selected_other_treshold = []
descriptions_to_export = []

genes_fams1 = []
genes_fams2 = []
genes_fams3 = []
genes_fams4 = []

In [8]:
for f in fams:
    fam = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-vep'+f+'.mt')
    fam = fam.annotate_cols(schema = schema[fam.s])
    fam = fam.annotate_rows(segregation = hl.agg.group_by(fam.schema.mutation_option_1, hl.agg.count_where(fam.GT.is_non_ref())))
    with_mutation = fam.aggregate_cols((hl.agg.counter(fam.schema.mutation_option_1))['YES'])
    fam = fam.filter_rows((fam.segregation.get('NO', 0) == 0) & (fam.segregation.get('YES', 0) == with_mutation))
    
    fam = fam.filter_rows(hl.is_snp(fam.alleles[0], fam.alleles[1]))
    
    fam1 = fam.filter_rows(
        (fam.cadd.score_phred > 10) &
        (hl.if_else(
            hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]), 
            fam.gnomad_v_3_1.freq.AF[2] < 0.05,
            True)
        )
    )
    
    fam3 = fam.filter_rows(
        hl.if_else(
            hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]),
            fam.gnomad_v_3_1.freq.AF[2] < 0.01,
            True
        )
    )

    genes_fams1.append(fam1.within_gene.collect())
    genes_fams3.append(fam3.within_gene.collect())
    
    fams1.append(fam1)
    fams3.append(fam3)
    
    fam2 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))
    fam4 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))
    
    fam2 = fam2.filter_rows(
        (fam2.cadd.score_phred > 10) &
        (hl.if_else(
            hl.is_defined(fam2.gnomad_v_3_1.freq.AF[2]), 
            fam2.gnomad_v_3_1.freq.AF[2] < 0.05,
            True)
        )
    )
    fam4 = fam4.filter_rows(
        hl.if_else(
            hl.is_defined(fam4.gnomad_v_3_1.freq.AF[2]),
            fam4.gnomad_v_3_1.freq.AF[2] < 0.1,
            True
        )
    )


    genes_fams2.append(fam2.within_gene.collect())
    genes_fams4.append(fam4.within_gene.collect())
    
    fams2.append(fam2)
    fams4.append(fam4)

In [10]:
#add H1, H2 and T1, T2 options:

fams1_h = []
fams2_h = []
fams3_h = []
fams4_h = []

genes_fams1_h = []
genes_fams2_h = []
genes_fams3_h = []
genes_fams4_h = []

fam = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-vepH.mt')
fam = fam.annotate_cols(schema = schema[fam.s])
fam = fam.annotate_rows(segregation = hl.agg.group_by(fam.schema.mutation_option_1, hl.agg.count_where(fam.GT.is_non_ref())))
with_mutation = fam.aggregate_cols((hl.agg.counter(fam.schema.mutation_option_1))['YES'])
fam = fam.filter_rows((fam.segregation.get('NO', 0) == 0) & (fam.segregation.get('YES', 0) == with_mutation))
fam = fam.filter_rows(hl.is_snp(fam.alleles[0], fam.alleles[1]))

fam1 = fam.filter_rows(
        (fam.cadd.score_phred > 10) &
        (hl.if_else(
            hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]), 
            fam.gnomad_v_3_1.freq.AF[2] < 0.05,
            True)
        )
)
    
fam3 = fam.filter_rows(
        hl.if_else(
            hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]),
            fam.gnomad_v_3_1.freq.AF[2] < 0.01,
            True
        )
)

genes_fams1_h.append(fam1.within_gene.collect())
genes_fams3_h.append(fam3.within_gene.collect())

fams1_h.append(fam1)
fams3_h.append(fam3)
   
fam2 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))
fam4 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))

fam2 = fam2.filter_rows(
        (fam2.cadd.score_phred > 10) &
        (hl.if_else(
            hl.is_defined(fam2.gnomad_v_3_1.freq.AF[2]), 
            fam2.gnomad_v_3_1.freq.AF[2] < 0.05,
            True)
        )
    )
fam4 = fam4.filter_rows(
        hl.if_else(
            hl.is_defined(fam4.gnomad_v_3_1.freq.AF[2]),
            fam4.gnomad_v_3_1.freq.AF[2] < 0.1,
            True
        )
    )

genes_fams2_h.append(fam2.within_gene.collect())
fams2_h.append(fam2)

genes_fams4_h.append(fam4.within_gene.collect())
fams4_h.append(fam4)


fam = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-vepH2.mt')
fam = fam.annotate_cols(schema = schema[fam.s])
fam = fam.annotate_rows(segregation = hl.agg.group_by(fam.schema.mutation_option_2, hl.agg.count_where(fam.GT.is_non_ref())))
with_mutation = fam.aggregate_cols((hl.agg.counter(fam.schema.mutation_option_2))['YES'])
fam = fam.filter_rows((fam.segregation.get('NO', 0) == 0) & (fam.segregation.get('YES', 0) == with_mutation))    
fam = fam.filter_rows(hl.is_snp(fam.alleles[0], fam.alleles[1]))
    
fam1 = fam.filter_rows(
        (fam.cadd.score_phred > 10) &
        (hl.if_else(
            hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]), 
            fam.gnomad_v_3_1.freq.AF[2] < 0.001,
            True)
        )
)
    
fam3 = fam.filter_rows(
        hl.if_else(
            hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]),
            fam.gnomad_v_3_1.freq.AF[2] < 0.01,
            True
        )
)

genes_fams1_h.append(fam1.within_gene.collect())
genes_fams3_h.append(fam3.within_gene.collect())

fams1_h.append(fam1)
fams3_h.append(fam3)
   
fam2 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))
fam4 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))

fam2 = fam2.filter_rows(
        (fam2.cadd.score_phred > 10) &
        (hl.if_else(
            hl.is_defined(fam2.gnomad_v_3_1.freq.AF[2]), 
            fam2.gnomad_v_3_1.freq.AF[2] < 0.05,
            True)
        )
    )
fam4 = fam4.filter_rows(
        hl.if_else(
            hl.is_defined(fam4.gnomad_v_3_1.freq.AF[2]),
            fam4.gnomad_v_3_1.freq.AF[2] < 0.1,
            True
        )
    )

genes_fams2_h.append(fam2.within_gene.collect())
genes_fams4_h.append(fam4.within_gene.collect())

fams4_h.append(fam4)
fams2_h.append(fam2)


fams1_t = []
fams2_t = []
fams3_t = []
fams4_t = []

genes_fams1_t = []
genes_fams2_t = []
genes_fams3_t = []
genes_fams4_t = []

fam = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-vepT.mt')
fam = fam.annotate_cols(schema = schema[fam.s])
fam = fam.annotate_rows(segregation = hl.agg.group_by(fam.schema.mutation_option_1, hl.agg.count_where(fam.GT.is_non_ref())))
with_mutation = fam.aggregate_cols((hl.agg.counter(fam.schema.mutation_option_1))['YES'])
fam = fam.filter_rows((fam.segregation.get('NO', 0) == 0) & (fam.segregation.get('YES', 0) == with_mutation))
fam = fam.filter_rows(hl.is_snp(fam.alleles[0], fam.alleles[1]))
    
fam1 = fam.filter_rows(
        (fam.cadd.score_phred > 10) &
        (hl.if_else(
            hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]), 
            fam.gnomad_v_3_1.freq.AF[2] < 0.001,
            True)
        )
)
    
fam3 = fam.filter_rows(
        hl.if_else(
            hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]),
            fam.gnomad_v_3_1.freq.AF[2] < 0.01,
            True
        )
)

genes_fams1_t.append(fam1.within_gene.collect())
genes_fams3_t.append(fam3.within_gene.collect())

fams1_t.append(fam1)
fams3_t.append(fam3)
   
fam2 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))
fam4 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))

fam2 = fam2.filter_rows(
        (fam2.cadd.score_phred > 10) &
        (hl.if_else(
            hl.is_defined(fam2.gnomad_v_3_1.freq.AF[2]), 
            fam2.gnomad_v_3_1.freq.AF[2] < 0.05,
            True)
        )
    )
fam4 = fam4.filter_rows(
        hl.if_else(
            hl.is_defined(fam4.gnomad_v_3_1.freq.AF[2]),
            fam4.gnomad_v_3_1.freq.AF[2] < 0.1,
            True
        )
    )

genes_fams2_t.append(fam2.within_gene.collect())
genes_fams4_t.append(fam4.within_gene.collect())

fams4_t.append(fam4)
fams2_t.append(fam2)

fam = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-vepT2.mt')
fam = fam.annotate_cols(schema = schema[fam.s])
fam = fam.annotate_rows(segregation = hl.agg.group_by(fam.schema.mutation_option_2, hl.agg.count_where(fam.GT.is_non_ref())))
with_mutation = fam.aggregate_cols((hl.agg.counter(fam.schema.mutation_option_2))['YES'])
fam = fam.filter_rows((fam.segregation.get('NO', 0) == 0) & (fam.segregation.get('YES', 0) == with_mutation))    
fam = fam.filter_rows(hl.is_snp(fam.alleles[0], fam.alleles[1]))
    
fam1 = fam.filter_rows(
        (fam.cadd.score_phred > 10) &
        (hl.if_else(
            hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]), 
            fam.gnomad_v_3_1.freq.AF[2] < 0.05,
            True)
        )
)
    
fam3 = fam.filter_rows(
        hl.if_else(
            hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]),
            fam.gnomad_v_3_1.freq.AF[2] < 0.01,
            True
        )
)

genes_fams1_t.append(fam1.within_gene.collect())
genes_fams3_t.append(fam3.within_gene.collect())

fams1_t.append(fam1)
fams3_t.append(fam3)
   
fam2 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))
fam4 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))

fam2 = fam2.filter_rows(
        (fam2.cadd.score_phred > 10) &
        (hl.if_else(
            hl.is_defined(fam2.gnomad_v_3_1.freq.AF[2]), 
            fam2.gnomad_v_3_1.freq.AF[2] < 0.05,
            True)
        )
    )
fam4 = fam4.filter_rows(
        hl.if_else(
            hl.is_defined(fam4.gnomad_v_3_1.freq.AF[2]),
            fam4.gnomad_v_3_1.freq.AF[2] < 0.1,
            True
        )
    )

genes_fams2_t.append(fam2.within_gene.collect())
genes_fams4_t.append(fam4.within_gene.collect())

fams4_t.append(fam4)
fams2_t.append(fam2)

In [11]:
len(fams1)

15

In [43]:
### This is an optional step to get only variants that are duplicated

for idx, gene_list in enumerate(genes_fams1_h):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams1_h[idx] = list(set(gene_list))

for idx, gene_list in enumerate(genes_fams1_t):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams1_t[idx] = list(set(gene_list))

for idx, gene_list in enumerate(genes_fams1):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams1[idx] = list(set(gene_list))

for idx, gene_list in enumerate(genes_fams3_h):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams3_h[idx] = list(set(gene_list))

for idx, gene_list in enumerate(genes_fams3_t):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams3_t[idx] = list(set(gene_list))

for idx, gene_list in enumerate(genes_fams3):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams3[idx] = list(set(gene_list))

genes_fams1_h = [item for sublist in genes_fams1_h for item in sublist] 
genes_fams1_t = [item for sublist in genes_fams1_t for item in sublist]
genes_fams1 = [item for sublist in genes_fams1 for item in sublist]

genes_fams3_h = [item for sublist in genes_fams3_h for item in sublist] 
genes_fams3_t = [item for sublist in genes_fams3_t for item in sublist]
genes_fams3 = [item for sublist in genes_fams3 for item in sublist]

# now get only unique items from genes_fams1_h and genes_fams1_t:
genes_fams1_h = list(set(genes_fams1_h))
genes_fams1_t = list(set(genes_fams1_t))

genes_fams3_h = list(set(genes_fams3_h))
genes_fams3_t = list(set(genes_fams3_t))

In [44]:
genes_fams1 = genes_fams1 + genes_fams1_h + genes_fams1_t
genes_fams3 = genes_fams3 + genes_fams3_h + genes_fams3_t

In [45]:
seen = set()
dupes_1 = [x for x in genes_fams1 if x in seen or seen.add(x)]    
len(set(dupes_1)) #this should contain all the genes that repeat themselfs excluding genes common for H1 and H2 and T1 and T2

122

In [46]:
seen = set()
dupes_3 = [x for x in genes_fams3 if x in seen or seen.add(x)]    
len(set(dupes_3)) #this should contain all the genes that repeat themselfs excluding genes common for H1 and H2 and T1 and T2

1575

In [47]:
for idx, gene_list in enumerate(genes_fams2_h):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams2_h[idx] = list(set(gene_list))

for idx, gene_list in enumerate(genes_fams2_t):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams2_t[idx] = list(set(gene_list))

for idx, gene_list in enumerate(genes_fams2):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams2[idx] = list(set(gene_list))

for idx, gene_list in enumerate(genes_fams4_h):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams4_h[idx] = list(set(gene_list))

for idx, gene_list in enumerate(genes_fams4_t):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams4_t[idx] = list(set(gene_list))

for idx, gene_list in enumerate(genes_fams4):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family and remove variants that are double within a family
    genes_fams4[idx] = list(set(gene_list))

In [48]:
genes_fams2_h = [item for sublist in genes_fams2_h for item in sublist] 
genes_fams2_t = [item for sublist in genes_fams2_t for item in sublist]
genes_fams2 = [item for sublist in genes_fams2 for item in sublist]
genes_fams4_h = [item for sublist in genes_fams4_h for item in sublist] 
genes_fams4_t = [item for sublist in genes_fams4_t for item in sublist]
genes_fams4 = [item for sublist in genes_fams4 for item in sublist]

In [49]:
# now get only unique items from genes_fams1_h and genes_fams1_t:
genes_fams2_h = list(set(genes_fams2_h))
genes_fams2_t = list(set(genes_fams2_t))
genes_fams2 = genes_fams2 + genes_fams2_h + genes_fams2_t

genes_fams4_h = list(set(genes_fams4_h))
genes_fams4_t = list(set(genes_fams4_t))
genes_fams4 = genes_fams4 + genes_fams4_h + genes_fams4_t

In [50]:
seen = set()
dupes_2 = [x for x in genes_fams2 if x in seen or seen.add(x)]    
len(set(dupes_2)) #this should contain all the genes that repeat themselfs excluding genes common for H1 and H2 and T1 and T2

80

In [51]:
seen = set()
dupes_4 = [x for x in genes_fams4 if x in seen or seen.add(x)]    
len(set(dupes_4)) #this should contain all the genes that repeat themselfs excluding genes common for H1 and H2 and T1 and T2

335

In [12]:
# now append h1 h2 and t1 t2 to fams1
fams.append('H1')
fams.append('H2')
fams_all_genes.append('H1_all_genes')
fams_all_genes.append('H2_all_genes')
fams_selected_genes.append('H1_selected_genes')
fams_selected_genes.append('H2_selected_genes')
fams_all_genes_other_treshold.append('H1_all_genes_lower_treshold')
fams_all_genes_other_treshold.append('H2_all_genes_lower_treshold')
fams_selected_genes_other_treshold.append('H1_selected_genes_lower_treshold')
fams_selected_genes_other_treshold.append('H2_selected_genes_lower_treshold')
fams_description.append('H1_description')
fams_description.append('H2_description')

fams.append('T1')
fams.append('T2')
fams_all_genes.append('T1_all_genes')
fams_all_genes.append('T2_all_genes')
fams_selected_genes.append('T1_selected_genes')
fams_selected_genes.append('T2_selected_genes')
fams_all_genes_other_treshold.append('T1_all_genes_lower_treshold')
fams_all_genes_other_treshold.append('T2_all_genes_lower_treshold')
fams_selected_genes_other_treshold.append('T1_selected_genes_lower_treshold')
fams_selected_genes_other_treshold.append('T2_selected_genes_lower_treshold')
fams_description.append('T1_description')
fams_description.append('T2_description')

fams1 = fams1 + fams1_h + fams1_t
fams2 = fams2 + fams2_h + fams2_t
fams3 = fams3 + fams3_h + fams3_t
fams4 = fams4 + fams4_h + fams4_t

In [13]:
len(fams1)

19

In [14]:
len(fams2)

19

In [15]:
len(fams3)

19

In [16]:
len(fams4)

19

In [53]:
#now filter all the families only to have variants with duplicated genes
for idx, fam in enumerate(fams1):
    fam = fam.filter_rows((hl.any(lambda x: hl.literal(dupes_1).contains(x), fam.within_gene)))
    fams1[idx] = fam

for idx, fam in enumerate(fams2):
    fam = fam.filter_rows((hl.any(lambda x: hl.literal(dupes_2).contains(x), fam.within_gene)))
    fams2[idx] = fam
    
for idx, fam in enumerate(fams3):
    fam = fam.filter_rows((hl.any(lambda x: hl.literal(dupes_3).contains(x), fam.within_gene)))
    fams3[idx] = fam

for idx, fam in enumerate(fams4):
    fam = fam.filter_rows((hl.any(lambda x: hl.literal(dupes_4).contains(x), fam.within_gene)))
    fams4[idx] = fam

In [17]:
for fam in fams1:
    fam = fam.drop(fam['a_index'], fam['was_split'])
    #also annotate rows with family
    fam = fam.transmute_rows(DP_stats = fam.dp_qc,
                             GQ_stats =fam.gq_qc, 
                             gnomad_v3_nfe_af = fam.gnomad_v_3_1.freq.AF[2],
                             gnomad_v3_nfe_homozygote_count = fam.gnomad_v_3_1.freq.homozygote_count[2],
                             cadd = fam.cadd.score_phred,
                             within_gene = fam.within_gene,
                             hpo = fam.hpo,
                             non_refs_healthy = fam.segregation.get('NO', 0),
                             non_refs_gts = fam.segregation.get('YES', 0),
                             most_severe_consequence = fam.vep.vep.most_severe_consequence,
                             transcript_consequences = fam.vep.vep.transcript_consequences,
                             intergenic_consequences = fam.vep.vep.intergenic_consequences,
                             motif_feature_consequences = fam.vep.vep.motif_feature_consequences,
                             regulatory_feature_consequences = fam.vep.vep.regulatory_feature_consequences,
                             family_non_ref = hl.agg.filter(hl.any(fam.GT.is_non_ref()), hl.agg.collect(fam.phenotypes.family))[0])
    fam = fam.select_entries(fam.GT)
    #fam.make_table().export('/net/scratch/people/plggosborcz/temp-mts/temp.csv', delimiter='\t')
    fam.rows().export('/net/scratch/people/plggosborcz/temp-mts/temp.csv', delimiter='\t')
    fams_to_export_all.append(pd.read_csv('/net/scratch/people/plggosborcz/temp-mts/temp.csv', sep ='\t')) 

2022-05-21 20:48:06 Hail: INFO: merging 51 files totalling 3.9M...(49 + 1) / 50]
2022-05-21 20:48:06 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 388.133ms
2022-05-21 20:48:14 Hail: INFO: merging 51 files totalling 396.3K...9 + 1) / 50]
2022-05-21 20:48:14 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 51.441ms
2022-05-21 20:48:22 Hail: INFO: merging 51 files totalling 238.8K...0 + 0) / 50]
2022-05-21 20:48:22 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 47.101ms
2022-05-21 20:48:30 Hail: INFO: merging 51 files totalling 1.6M...(49 + 1) / 50]
2022-05-21 20:48:30 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 52.085ms
2022-05-21 20:48:38 Hail: INFO: merging 51 files totalling 2.1M...(48 + 2) / 50]
2022-05-21 20:48:38 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.c

In [18]:
for fam in fams1:
    descriptions_to_export.append(fam.cols().to_pandas())

2022-05-21 20:50:29 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2022-05-21 20:50:30 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:31 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:31 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:32 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:33 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:34 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:34 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:35 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:36 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:37 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:37 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:38 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:39 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:39 Hail: INFO: Coerced sorted dataset
2022-05-21 20:50:40 Hail: INFO: Coerced sorted dataset
2022

In [19]:
for fam in fams3:
    fam = fam.drop(fam['a_index'], fam['was_split'])
    #also annotate rows with family
    fam = fam.transmute_rows(DP_stats = fam.dp_qc,
                             GQ_stats =fam.gq_qc, 
                             gnomad_v3_nfe_af = fam.gnomad_v_3_1.freq.AF[2],
                             gnomad_v3_nfe_homozygote_count = fam.gnomad_v_3_1.freq.homozygote_count[2],
                             cadd = fam.cadd.score_phred,
                             within_gene = fam.within_gene,
                             hpo = fam.hpo,
                             non_refs_healthy = fam.segregation.get('NO', 0),
                             non_refs_gts = fam.segregation.get('YES', 0),
                             most_severe_consequence = fam.vep.vep.most_severe_consequence,
                             transcript_consequences = fam.vep.vep.transcript_consequences,
                             intergenic_consequences = fam.vep.vep.intergenic_consequences,
                             motif_feature_consequences = fam.vep.vep.motif_feature_consequences,
                             regulatory_feature_consequences = fam.vep.vep.regulatory_feature_consequences,
                             family_non_ref = hl.agg.filter(hl.any(fam.GT.is_non_ref()), hl.agg.collect(fam.phenotypes.family))[0])
    fam = fam.select_entries(fam.GT)
    #fam.make_table().export('/net/scratch/people/plggosborcz/temp-mts/temp.csv', delimiter='\t')
    fam.rows().export('/net/scratch/people/plggosborcz/temp-mts/temp.csv', delimiter='\t')
    fams_to_export_all_other_treshold.append(pd.read_csv('/net/scratch/people/plggosborcz/temp-mts/temp.csv', sep ='\t')) 

2022-05-21 20:50:51 Hail: INFO: merging 51 files totalling 3.9M...(49 + 1) / 50]
2022-05-21 20:50:51 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 60.152ms
2022-05-21 20:50:59 Hail: INFO: merging 51 files totalling 348.3K...9 + 1) / 50]
2022-05-21 20:50:59 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 49.599ms
2022-05-21 20:51:07 Hail: INFO: merging 51 files totalling 163.9K...9 + 1) / 50]
2022-05-21 20:51:07 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 47.463ms
2022-05-21 20:51:14 Hail: INFO: merging 51 files totalling 1.5M...(49 + 1) / 50]
2022-05-21 20:51:15 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 55.251ms
2022-05-21 20:51:22 Hail: INFO: merging 51 files totalling 2.0M...(48 + 2) / 50]
2022-05-21 20:51:22 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.cs

In [20]:
for fam in fams2:
    fam = fam.drop(fam['a_index'], fam['was_split'])
    fam = fam.transmute_rows(DP_stats = fam.dp_qc,
                             GQ_stats =fam.gq_qc, 
                             gnomad_v3_nfe_af = fam.gnomad_v_3_1.freq.AF[2],
                             gnomad_v3_nfe_homozygote_count = fam.gnomad_v_3_1.freq.homozygote_count[2],
                             cadd = fam.cadd.score_phred,
                             within_gene = fam.within_gene,
                             hpo = fam.hpo,
                             non_refs_healthy = fam.segregation.get('NO', 0),
                             non_refs_gts = fam.segregation.get('YES', 0),
                             most_severe_consequence = fam.vep.vep.most_severe_consequence,
                             transcript_consequences = fam.vep.vep.transcript_consequences,
                             intergenic_consequences = fam.vep.vep.intergenic_consequences,
                             motif_feature_consequences = fam.vep.vep.motif_feature_consequences,
                             regulatory_feature_consequences = fam.vep.vep.regulatory_feature_consequences,
                             family_non_ref = hl.agg.filter(hl.any(fam.GT.is_non_ref()), hl.agg.collect(fam.phenotypes.family))[0])

    #fam.make_table().export('/net/scratch/people/plggosborcz/temp-mts/temp.csv', delimiter='\t')
    fam.rows().export('/net/scratch/people/plggosborcz/temp-mts/temp.csv', delimiter='\t') #this is for joint tables export
    fams_to_export_selected.append(pd.read_csv('/net/scratch/people/plggosborcz/temp-mts/temp.csv', sep ='\t')) 

2022-05-21 20:53:18 Hail: INFO: merging 51 files totalling 244.7K...9 + 1) / 50]
2022-05-21 20:53:18 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 48.665ms
2022-05-21 20:53:26 Hail: INFO: merging 51 files totalling 43.0K...49 + 1) / 50]
2022-05-21 20:53:26 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 47.153ms
2022-05-21 20:53:33 Hail: INFO: merging 51 files totalling 7.4K...(49 + 1) / 50]
2022-05-21 20:53:33 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 46.333ms
2022-05-21 20:53:41 Hail: INFO: merging 51 files totalling 179.8K...9 + 1) / 50]
2022-05-21 20:53:41 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 47.211ms
2022-05-21 20:53:48 Hail: INFO: merging 51 files totalling 183.2K...9 + 1) / 50]
2022-05-21 20:53:49 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.cs

In [21]:
for fam in fams4:
    fam = fam.drop(fam['a_index'], fam['was_split'])
    fam = fam.transmute_rows(DP_stats = fam.dp_qc,
                             GQ_stats =fam.gq_qc, 
                             gnomad_v3_nfe_af = fam.gnomad_v_3_1.freq.AF[2],
                             gnomad_v3_nfe_homozygote_count = fam.gnomad_v_3_1.freq.homozygote_count[2],
                             cadd = fam.cadd.score_phred,
                             within_gene = fam.within_gene,
                             hpo = fam.hpo,
                             non_refs_healthy = fam.segregation.get('NO', 0),
                             non_refs_gts = fam.segregation.get('YES', 0),
                             most_severe_consequence = fam.vep.vep.most_severe_consequence,
                             transcript_consequences = fam.vep.vep.transcript_consequences,
                             intergenic_consequences = fam.vep.vep.intergenic_consequences,
                             motif_feature_consequences = fam.vep.vep.motif_feature_consequences,
                             regulatory_feature_consequences = fam.vep.vep.regulatory_feature_consequences,
                             family_non_ref = hl.agg.filter(hl.any(fam.GT.is_non_ref()), hl.agg.collect(fam.phenotypes.family))[0])
    
    #fam.make_table().export('/net/scratch/people/plggosborcz/temp-mts/temp.csv', delimiter='\t')
    fam.rows().export('/net/scratch/people/plggosborcz/temp-mts/temp.csv', delimiter='\t') #this is for joint tables export
    fams_to_export_selected_other_treshold.append(pd.read_csv('/net/scratch/people/plggosborcz/temp-mts/temp.csv', sep ='\t')) 

2022-05-21 20:55:42 Hail: INFO: merging 51 files totalling 1.1M...(48 + 2) / 50]
2022-05-21 20:55:42 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 53.350ms
2022-05-21 20:55:50 Hail: INFO: merging 51 files totalling 73.8K...49 + 1) / 50]
2022-05-21 20:55:50 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 45.603ms
2022-05-21 20:55:57 Hail: INFO: merging 51 files totalling 52.4K...49 + 1) / 50]
2022-05-21 20:55:57 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 47.763ms
2022-05-21 20:56:05 Hail: INFO: merging 51 files totalling 735.0K...9 + 1) / 50]
2022-05-21 20:56:05 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.csv
  merge time: 49.860ms
2022-05-21 20:56:12 Hail: INFO: merging 51 files totalling 908.5K...7 + 3) / 50]
2022-05-21 20:56:12 Hail: INFO: while writing:
    /net/scratch/people/plggosborcz/temp-mts/temp.cs

In [22]:
#pd.concat(fams_to_export_selected[0:19]).to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/families-05-2022-schema-duplicated-selected.csv')
pd.concat(fams_to_export_all[0:19]).to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/familes-all-5-percent.csv')
pd.concat(fams_to_export_selected_other_treshold[0:19]).to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/familes-all-1-percent.csv')
#pd.concat(fams_to_export_all_other_treshold[0:19]).to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/families-05-2022-schema-duplicateds-all-other.csv')

#this gives concatenated info

In [ ]:
#get list of duplicated genes - optional
results_all_genes = pd.concat(fams_to_export_all[0:19])['within_gene']
results_all_genes = [item for sublist in results_all_genes for item in sublist] 
list(set(results_all_genes))

In [75]:
list_of_tables = [fams_to_export_all,
                  fams_to_export_all_other_treshold, 
                  fams_to_export_selected,
                  fams_to_export_selected_other_treshold,
                  descriptions_to_export]
concatenated_list = [val for tup in zip(*list_of_tables) for val in tup]

sheet_names = [fams_all_genes,
               fams_all_genes_other_treshold,
               fams_selected_genes,
               fams_selected_genes_other_treshold,
               fams_description]

concatenated_list_sheets = [val for tup in zip(*sheet_names) for val in tup]

with pd.ExcelWriter('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/families-05-2022-schema.xlsx') as writer:
    for idx, res in enumerate(concatenated_list):
        res.to_excel(writer, sheet_name=str(concatenated_list_sheets[idx]), header=True, index=False)

## 7. Find variants that are common across multiple families

In [ ]:

loci_all = []
loci_selected = []

for f in fams_to_export_all:
    loci_all.append(list(f['locus.contig']+':'+f['locus.position'].astype(str)))
    f['locus'] = f['locus.contig']+':'+f['locus.position'].astype(str)
    
for f in fams_to_export_selected:
    loci_selected.append(list(f['locus.contig']+':'+f['locus.position'].astype(str)))
    f['locus'] = f['locus.contig']+':'+f['locus.position'].astype(str)

loci_all = [item for sublist in loci_all for item in sublist]
loci_selected = [item for sublist in loci_selected for item in sublist]

seen = set()
dupes_loci_all = [x for x in loci_all if x in seen or seen.add(x)]    

seen = set()
dupes_loci_selected = [x for x in loci_selected if x in seen or seen.add(x)]    

In [ ]:
seen = set()
dupes_loci_all = [x for x in loci_all if x in seen or seen.add(x)]    

In [ ]:
dupes_loci_all

In [ ]:
fams_to_export_all_filtered = []

for f in fams_to_export_all:
    f_filtered = f[f.locus.isin(dupes_loci_all)]
    fams_to_export_all_filtered.append(f_filtered)
    
fams_to_export_selected_filtered = []

for f in fams_to_export_selected:
    f_filtered = f[f.locus.isin(dupes_loci_selected)]
    fams_to_export_selected_filtered.append(f_filtered)

In [ ]:
list_of_tables = [fams_to_export_all_filtered, fams_to_export_selected_filtered, descriptions_to_export]
concatenated_list = [val for tup in zip(*list_of_tables) for val in tup]

sheet_names = [fams_all_genes, fams_selected_genes, fams_description]
concatenated_list_sheets = [val for tup in zip(*sheet_names) for val in tup]

with pd.ExcelWriter('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/families-11-2021-schema-duplicated-variants.xlsx') as writer:
    for idx, res in enumerate(concatenated_list):
        res.to_excel(writer, sheet_name=str(concatenated_list_sheets[idx]), header=True, index=False)

### some extra code (variant overrepresentation)

In [ ]:
fams = ['A', 'B', 'C','D', 'E', 'F', 'G', 'I', 'J', 'R', 'S', 'U', 'W', 'X', 'Y'] # h and t fams are excluded as they have two schemas of heredity
fams_all_genes = ['A_all_genes', 'B_all_genes', 'C_all_genes','D_all_genes',
                  'E_all_genes', 'F_all_genes', 'G_all_genes',
                  'I_all_genes', 'J_all_genes', 'R_all_genes', 'S_all_genes',
                  'U_all_genes', 'W_all_genes', 'X_all_genes', 'Y_all_genes']
fams_selected_genes = ['A_selected_genes', 'B_selected_genes', 'C_selected_genes','D_selected_genes',
                  'E_selected_genes', 'F_selected_genes', 'G_selected_genes',
                  'I_selected_genes', 'J_selected_genes', 'R_selected_genes', 'S_selected_genes',
                  'U_selected_genes', 'W_selected_genes', 'X_selected_genes', 'Y_selected_genes']
fams_description = ['A_description', 'B_description', 'C_description','D_description',
                  'E_description', 'F_description', 'G_description',
                  'I_description', 'J_description', 'R_description', 'S_description',
                  'U_description', 'W_description', 'X_description', 'Y_description']

fams1 = []
fams2 = []
fams_to_export_all = []
fams_to_export_selected = []
descriptions_to_export = []

genes_fams1 = []
genes_fams2 = []

for f in fams:
    
    fam = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fam-vep'+f+'.mt')
    ctrl = fam.aggregate_cols(hl.agg.counter(fam.phenotypes.disease))['NO']
    gts = fam.aggregate_cols(hl.agg.counter(fam.phenotypes.disease))['YES']

    fam = fam.annotate_rows(segregation = hl.agg.group_by(fam.phenotypes.disease, hl.agg.count_where(fam.GT.is_non_ref())))
    fam = fam.filter_rows(((fam.segregation.get('NO', 0))/ctrl < 0.33) & ((fam.segregation.get('YES', 0))/gts > 0.66))
    
    fam = fam.filter_rows(hl.is_snp(fam.alleles[0], fam.alleles[1]))
    
    fam1 = fam.filter_rows(hl.if_else(hl.is_defined(fam.gnomad_v_3_1.freq.AF[2]), fam.gnomad_v_3_1.freq.AF[2] < 0.001, True))
    genes_fams1.append(fam1.within_gene.collect())
    fams1.append(fam1)
    
    fam2 = fam.filter_rows((hl.any(lambda x: hl.literal(all_genes).contains(x), fam.within_gene)))
    fam2 = fam2.filter_rows(hl.if_else(hl.is_defined(fam2.gnomad_v_3_1.freq.AF[2]), fam2.gnomad_v_3_1.freq.AF[2] < 0.05, True))
    genes_fams2.append(fam2.within_gene.collect())
    fams2.append(fam2)

for idx, gene_list in enumerate(genes_fams1):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family
    genes_fams1[idx] = list(set(gene_list))

genes_fams1 = [item for sublist in genes_fams1 for item in sublist]

seen = set()
dupes_1 = [x for x in genes_fams1 if x in seen or seen.add(x)]    

len(set(dupes_1))

# optional - export only genes with multiple variants:

for idx, fam in enumerate(fams1):
    fam = fam.filter_rows((hl.any(lambda x: hl.literal(dupes_1).contains(x), fam.within_gene)))
    fams1[idx] = fam

for idx, gene_list in enumerate(genes_fams2):
    gene_list = [item for sublist in gene_list for item in sublist] # first flatten the gene list in each family
    genes_fams2[idx] = list(set(gene_list))

genes_fams2 = [item for sublist in genes_fams2 for item in sublist]

len(set(genes_fams2))

seen = set()
dupes_2 = [x for x in genes_fams2 if x in seen or seen.add(x)]    

len(set(dupes_2))

for idx, fam in enumerate(fams2):
    fam = fam.filter_rows((hl.any(lambda x: hl.literal(dupes_2).contains(x), fam.within_gene)))
    fams2[idx] = fam

for fam in fams1:
    fam = fam.drop(fam['a_index'], fam['was_split'])
    fam = fam.transmute_rows(DP_stats = fam.dp_qc,
                             GQ_stats =fam.gq_qc, 
                             gnomad_v3_nfe_af = fam.gnomad_v_3_1.freq.AF[2],
                             gnomad_v3_nfe_homozygote_count = fam.gnomad_v_3_1.freq.homozygote_count[2],
                             cadd = fam.cadd.cadd_score,
                             within_gene = fam.within_gene,
                             hpo = fam.hpo,
                             non_refs_healthy = fam.segregation.get('NO', 0),
                             non_refs_gts = fam.segregation.get('YES', 0),
                             most_severe_consequence = fam.vep.vep.most_severe_consequence,
                             transcript_consequences = fam.vep.vep.transcript_consequences,
                             intergenic_consequences = fam.vep.vep.intergenic_consequences,
                             motif_feature_consequences = fam.vep.vep.motif_feature_consequences,
                             regulatory_feature_consequences = fam.vep.vep.regulatory_feature_consequences)

    fam = fam.select_entries(fam.GT)
    fams_to_export_all.append(fam.make_table().to_pandas())
    descriptions_to_export.append(fam.cols().to_pandas())

for idx, f in enumerate(fams_to_export_all):
    fams_to_export_all[idx] = f[f.columns.drop(list(f.filter(regex='.phased')))]
    
    

for fam in fams2:
    fam = fam.drop(fam['a_index'], fam['was_split'])
    fam = fam.transmute_rows(DP_stats = fam.dp_qc,
                             GQ_stats =fam.gq_qc, 
                             gnomad_v3_nfe_af = fam.gnomad_v_3_1.freq.AF[2],
                             gnomad_v3_nfe_homozygote_count = fam.gnomad_v_3_1.freq.homozygote_count[2],
                             cadd = fam.cadd.cadd_score,
                             within_gene = fam.within_gene,
                             hpo = fam.hpo,
                             non_refs_healthy = fam.segregation.get('NO', 0),
                             non_refs_gts = fam.segregation.get('YES', 0),
                             most_severe_consequence = fam.vep.vep.most_severe_consequence,
                             transcript_consequences = fam.vep.vep.transcript_consequences,
                             intergenic_consequences = fam.vep.vep.intergenic_consequences,
                             motif_feature_consequences = fam.vep.vep.motif_feature_consequences,
                             regulatory_feature_consequences = fam.vep.vep.regulatory_feature_consequences)
    
    fam = fam.select_entries(fam.GT)
    fams_to_export_selected.append(fam.make_table().to_pandas())

for idx, f in enumerate(fams_to_export_selected):
    fams_to_export_selected[idx] = f[f.columns.drop(list(f.filter(regex='.phased')))]

list_of_tables = [fams_to_export_all, fams_to_export_selected, descriptions_to_export]
concatenated_list = [val for tup in zip(*list_of_tables) for val in tup]

sheet_names = [fams_all_genes, fams_selected_genes, fams_description]
concatenated_list_sheets = [val for tup in zip(*sheet_names) for val in tup]

with pd.ExcelWriter('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/families-11-2021-duplicate-genes.xlsx') as writer:
    for idx, res in enumerate(concatenated_list):
        res.to_excel(writer, sheet_name=str(concatenated_list_sheets[idx]), header=True, index=False)

loci_all = []
loci_selected = []

for f in fams_to_export_all:
    loci_all.append(list(f['locus.contig']+':'+f['locus.position'].astype(str)))
    f['locus'] = f['locus.contig']+':'+f['locus.position'].astype(str)
    
for f in fams_to_export_selected:
    loci_selected.append(list(f['locus.contig']+':'+f['locus.position'].astype(str)))
    f['locus'] = f['locus.contig']+':'+f['locus.position'].astype(str)

loci_all = [item for sublist in loci_all for item in sublist]
loci_selected = [item for sublist in loci_selected for item in sublist]

seen = set()
dupes_loci_all = [x for x in loci_all if x in seen or seen.add(x)]    

seen = set()
dupes_loci_selected = [x for x in loci_all if x in seen or seen.add(x)]    

fams_to_export_all_filtered = []

for f in fams_to_export_all:
    f_filtered = f[f.locus.isin(dupes_loci_all)]
    fams_to_export_all_filtered.append(f_filtered)
    
fams_to_export_selected_filtered = []

for f in fams_to_export_selected:
    f_filtered = f[f.locus.isin(dupes_loci_selected)]
    fams_to_export_selected_filtered.append(f_filtered)

list_of_tables = [fams_to_export_all_filtered, fams_to_export_selected_filtered, descriptions_to_export]
concatenated_list = [val for tup in zip(*list_of_tables) for val in tup]

sheet_names = [fams_all_genes, fams_selected_genes, fams_description]
concatenated_list_sheets = [val for tup in zip(*sheet_names) for val in tup]

with pd.ExcelWriter('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/families-11-2021-duplicated-variants.xlsx') as writer:
    for idx, res in enumerate(concatenated_list):
        res.to_excel(writer, sheet_name=str(concatenated_list_sheets[idx]), header=True, index=False)

### Investigate all carriers of selected mutations:

In [3]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/fams-anno.mt')

In [4]:
mt.count()

(15760027, 124)

In [18]:
variants = ['chr19:53803962:G:A',
            'chr5:76482747:A:C',
            'chr4:109086168:T:A',
            'chr1:223300619:G:A',
            'chr12:120368947:C:A',
            'chr1:31727178:G:A',
            'chr12:10723010:G:T',
            'chr11:111020375:C:A',
            'chr6:101375096:A:G',
            'chr12:97272797:T:A',
            'chr7:713068:C:T',
            'chr1:121379326:G:A',
            'chr10:74875378:T:C',
            'chr6:17649263:G:A',
            'chr7:106112481:G:A',
            'chr9:38396880:C:T',
            'chr18:12702512:G:A',
            'chr12:2855156:C:T',
            'chr11:113697354:C:T',
            'chr1:117120939:G:T',
            'chr3:149073278:C:A',
            'chr4:81459515:C:T',
            'chr14:96264526:C:T',
            'chr3:53855341:G:A',
            'chr7:2569000:G:A',
            'chr10:94687805:T:A',
            'chr4:88417579:G:T',
            'chr17:69026976:C:T',
            'chr16:2317763:T:A',
            'chr3:132601106:C:T',
            'chr12:100396328:A:C',
            'chr12:101319592:G:A',
            'chr12:1885984:G:A',
            'chr3:49004877:C:T',
            'chr17:10631666:T:C',
            'chr15:101324994:G:A',
            'chr16:4883689:G:A',
            'chr17:28385146:T:C',
            'chr16:89727323:C:T',
            'chr12:101677317:G:A',
            'chr12:111803962:G:A',
            'chr12:94279637:G:A',
            'chr12:57244143:G:A',
            'chr6:39861001:A:G',
            'chr8:99999384:G:A',
            'chr21:46114063:G:A',
            'chr2:108499560:A:G',
            'chr3:49652822:G:C',
            'chr16:80549649:C:G',
            'chr7:107916945:T:C',
            'chr9:37541696:G:A']

In [6]:
for idx,v in enumerate(variants):
    variants[idx] = hl.parse_variant(v, reference_genome='GRCh38')

In [7]:
variants_to_filter = hl.utils.range_table(len(variants))

In [8]:
variants_to_filter = variants_to_filter.annotate(
    locus = hl.literal(variants)[variants_to_filter.idx]['locus'],
    alleles = hl.literal(variants)[variants_to_filter.idx]['alleles']
)

In [9]:
variants_to_filter = variants_to_filter.key_by(
    variants_to_filter.locus, variants_to_filter.alleles
)

In [19]:
variants_to_filter.count()

51

In [11]:
mt = mt.filter_rows(hl.is_defined(variants_to_filter[mt.row_key]))

In [ ]:
mt.count()

2023-01-09 16:15:51.446 Hail: INFO: Ordering unsorted dataset with network shuffle


In [13]:
mt = mt.annotate_rows(
    samples_het = hl.agg.filter(mt.GT.is_het(), hl.agg.collect(mt.s)),
    samples_hom_var = hl.agg.filter(mt.GT.is_hom_var(), hl.agg.collect(mt.s))
    )

In [14]:
mt = mt.rows()

In [15]:
mt = mt.select(mt.samples_het, mt.samples_hom_var)

In [ ]:
mt.show()

In [ ]:
mt.export(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/results/burden-and-family/samples_non_ref.csv', delimiter='\t'
)
